In [16]:
import ephem

import adi

from multiprocess import Process

import numpy as np

import time

from ipywidgets import Label

from datetime import datetime, timedelta

import odrive
from odrive.enums import *

# Setup Dish Control

In [ ]:
odrv0 = odrive.find_any()

In [ ]:
print(str(odrv0.vbus_voltage))

In [ ]:
# Find the index of the Encoder for precice start up positioning.
# This presumes the dish is parked just past the index pulse as the homed revolution
odrv0.axis0.requested_state = AXIS_STATE_ENCODER_INDEX_SEARCH
odrv0.axis1.requested_state = AXIS_STATE_ENCODER_INDEX_SEARCH

In [ ]:
# Assumes we are sending commands at ~10Hz so set postion filter bandwidht to 1/2 that
odrv0.axis0.controller.config.input_filter_bandwidth = 5
odrv0.axis1.controller.config.input_filter_bandwidth = 5

In [ ]:
odrv0.clear_errors()
odrv0.axis0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
odrv0.axis0.controller.config.input_mode = INPUT_MODE_POS_FILTER

In [ ]:
# Axis 1 is not doing the derbin
odrv0.clear_errors()
odrv0.axis1.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE

In [ ]:
odrv0.clear_errors()
odrv0.axis1.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
odrv0.axis1.controller.config.input_mode = INPUT_MODE_POS_FILTER

## Test Movement
Run the cells below to check that each dish axis moves ~5 degrees and back

In [ ]:
odrv0.axis0.controller.input_pos = 5

In [ ]:
odrv0.axis0.encoder.pos_estimate

In [ ]:
odrv0.axis0.controller.input_pos = 0

In [ ]:
odrv0.axis0.encoder.pos_estimate

In [ ]:
odrv0.axis1.controller.input_pos = 5

In [ ]:
odrv0.axis1.encoder.pos_estimate

In [ ]:
odrv0.axis1.controller.input_pos = 0

In [ ]:
odrv0.axis1.encoder.pos_estimate

# Calculate Passes

In [17]:
dish = ephem.Observer()
dish.lon = '-105.251825'
dish.lat = '40.010886'
dish.elevation = 1605 # Meters

In [27]:
sats = [
    {
        'sat': ephem.readtle('CIRBE',
            '1 56188U 23054L   23299.28287952  .00028502  00000-0  10188-2 0  9997',
            '2 56188  97.3877 192.7599 0005978 295.1803  64.8817 15.28764848 30167'
        ),
        'freq': 2.402e9
    },
    {
        'sat': ephem.readtle('CUTE',
            '1 49263U 21088D   23298.90579824  .00010951  00000-0  64582-3 0  9995',
            '2 49263  97.5314   2.5893 0017260 169.6161 190.5429 15.11611011114013'
        ),
        'freq': 2.402e9
    },
    {
        'sat': ephem.readtle('CITM',
            '1 52950U 22074G   23290.56415723  .00215632  00000-0  21039-2 0  9991',
            '2 52950  44.9994  27.0917 0015040 333.8983  26.1155 15.64672021 72604'
        ),
        'freq': 2.402e9
    }
]

In [28]:
start_time = datetime.utcnow()
run_until = ephem.Date(start_time + timedelta(hours=12))
start_time = ephem.Date(start_time)
#start_time = ephem.Date(datetime(2023, 10, 26, 17, 0, 0, 0))
#run_until = ephem.Date(datetime(2023, 10, 26, 19, 0, 0, 0))

In [29]:
passes = []

min_pass_time = 5

for sat_data in sats:

    sat = sat_data['sat']

    set = ephem.Date(start_time)
    while True:

        dish.date = set
        nxt_ps = dish.next_pass(sat)
        rise = nxt_ps[0]
        rise_az = nxt_ps[1]
        set = nxt_ps[4]
        set_az = nxt_ps[5]

        if set > run_until:
            break

        pass_time_min = (set-rise)*24*60

        if pass_time_min < min_pass_time:
            continue
        
        passes.append({
            'sat': sat_data,
            'rise_time': rise,
            'rise_az': rise_az,
            'set_time': set,
            'set_az': set_az
        })

In [30]:
passes.sort(key=lambda x : x['rise_time'])

In [31]:
for xrs in passes:
    print(xrs['sat']['sat'].name, xrs['rise_time'])

CUTE 2023/10/26 18:54:51
CUTE 2023/10/27 02:30:24
CUTE 2023/10/27 04:00:26
CITM 2023/10/27 04:43:04
CIRBE 2023/10/27 05:18:10
CUTE 2023/10/27 05:35:56
CITM 2023/10/27 06:17:55


# Run System

In [ ]:
az_offset = np.radians(15.4)

In [5]:
def to_file_name(date):
    return str(date).replace('/', '').replace(':', '').replace(' ', '_')

In [9]:
def get_current_time():
    return datetime.utcnow()

def set_dish_pos(a0_tgt, a1_tgt):
    odrv0.axis0.controller.input_pos = a0_tgt
    odrv0.axis1.controller.input_pos = a1_tgt

def get_dish_pos():
    return odrv0.axis0.encoder.pos_estimate, odrv0.axis1.encoder.pos_estimate

def setup_sdr(center_freq):
    # We are tuning with signal offset tuned 500KHz in the USB

    sample_rate = 2e6 # Hz
    num_samps = 1000000 # number of samples returned per call to rx()
    
    sdr = adi.Pluto('ip:192.168.2.1')
    sdr.gain_control_mode_chan0 = 'manual'
    sdr.rx_hardwaregain_chan0 = 70.0 # dB
    sdr.rx_lo = int(center_freq - 500e3)
    sdr.sample_rate = int(sample_rate)
    sdr.rx_rf_bandwidth = int(sample_rate) # filter width, just set it to the same as sample rate for now
    sdr.rx_buffer_size = num_samps

def record(filename, stop_time):
    import adi
    from datetime import datetime, timedelta
    sdr = adi.Pluto('ip:192.168.2.1')
    with open(filename, 'wb') as f:
        while datetime.now() < stop_time:
            data = sdr.rx().astype(np.csingle)
            f.write(data.tobytes())
            f.flush()

def record_sdr(filename, stop_time):
    p = Process(target=record, args=(filename, stop_time))
    p.start()
    return p

## Test SDR (DON'T RUN)

In [ ]:
setup_sdr(2.402e9)

In [ ]:
record('test.iq', datetime.now() + timedelta(seconds=10))

## Stubs (DON'T RUN)

In [ ]:
offset = datetime.utcnow() - datetime(2023, 10, 18, 14, 26, 11, 0) - timedelta(seconds=-10)

In [ ]:
def get_current_time():
    return datetime.utcnow() - offset

def set_dish_pos(a0_tgt, a1_tgt):
    pass

def get_dish_pos():
    return 0, 0

def setup_sdr(center_freq):
    print(center_freq)

def record_sdr(filename, stop_time):
    pass

## Start system

In [ ]:
el_label = Label()
display(el_label)

In [3]:
from types import SimpleNamespace
xrs = {
    'rise_time': ephem.Date(datetime.now()),
    'set_time': ephem.Date(datetime.now())
}
sat = SimpleNamespace()
sat.name = 'satname'
sat_data = {
    'freq': 2.4e9
}

In [10]:
filename = f"{sat.name}_{to_file_name(xrs['rise_time'])}.iq"
setup_sdr(sat_data['freq'])
p = record_sdr(filename, ephem.localtime(xrs['set_time']))
p.join()

In [ ]:
for xrs in passes:

    sat_data = xrs['sat']
    sat = sat_data['sat']

    # Wait for pass start
    while ephem.Date(get_current_time()) < xrs['rise_time']:
        el_label.value = f"Waiting... {xrs['rise_time']}"
        time.sleep(1)

    pass_start_elevation = 0
    
    dish.date = get_current_time()
    sat.compute(dish)
    
    el_label.value = f'{np.degrees(sat.alt):3.2f} EL {np.degrees(sat.az):3.2f} AZ F'

    # Wait for satellite to come above horizon
    while sat.alt < np.radians(pass_start_elevation):
    
        time.sleep(1)
        dish.date = get_current_time()
        sat.compute(dish)
    
        el_label.value = f'{np.degrees(sat.alt):3.2f} EL {np.degrees(sat.az):3.2f} AZ W'

    times = []
    az_tgt = []
    el_tgt = []
    az_meas = []
    el_meas = []

    # Start Recording
    filename = f"{sat.name}_{to_file_name(xrs['rise_time'])}.iq"
    setup_sdr(sat_data['freq'])
    p = record_sdr(filename, ephem.localtime(xrs['set_time']))

    # Track satellite and record data
    while sat.alt >= np.radians(pass_start_elevation):
    
        current_time = get_current_time()
        dish.date = current_time
        sat.compute(dish)
    
        sat_az = sat.az + az_offset
        sat_el = sat.alt
    
        el_label.value = f'{np.degrees(sat_el):3.2f} EL {np.degrees(sat_az):3.2f} AZ R'
    
        # Handle crossing 360
        if len(az_tgt) == 0:
            az_last = np.degrees(sat_az)
        else:
            az_last = az_tgt[-1]
        az_next = np.degrees(sat_az)
        az_diff = az_next - az_last
        az_diff = (((az_diff + 180) % 360 ) - 180)
    
        az_tgt.append(az_last + az_diff)
        el_tgt.append(np.degrees(sat_el))

        a0_tgt = (az_tgt[-1] - 180) * (3199 / 3600)
        a1_tgt = (90 - el_tgt[-1]) * (3199 / 3600)

        set_dish_pos(a0_tgt, a1_tgt)

        a0_pos, a1_pos = get_dish_pos()
    
        az_meas.append( (a0_pos /  (3199 / 3600) ) + 180 )
        el_meas.append( 90 - (a1_pos / (3199 / 3600) ) )
        
        times.append(current_time)

    p.join()

In [ ]:
odrv0.axis0.controller.input_pos = 0
odrv0.axis1.controller.input_pos = 0